# Тема "Методы сбора и обработки данных из сети Интернет"

### Урок 2. Парсинг HTML. BeautifulSoup

---
### Задание 1

Необходимо собрать информацию о вакансиях на вводимую должность (используем `input` или через аргументы получаем должность) с сайтов **hh.ru** (обязательно) и/или **superjob.ru** (по желанию). Приложение должно анализировать несколько страниц сайта (также вводим через `input` или аргументы). Получившийся список должен содержать в себе минимум:
1. Наименование вакансии.
2. Предлагаемую зарплату (разносим в три поля: минимальная и максимальная и валюта. цифры преобразуем к цифрам).
3. Ссылку на саму вакансию.
4. Сайт, откуда собрана вакансия.

По желанию можно добавить ещё параметры вакансии (например, работодателя и расположение). Структура должна быть одинаковая для вакансий с обоих сайтов. Общий результат можно вывести с помощью `dataFrame` через `pandas`. Сохраните в `json` либо `csv`.

In [1]:
import requests
from bs4 import BeautifulSoup as bs
import pandas as pd
import json

In [2]:
position = input('Введите искомую должность: ')

Введите искомую должность: python


In [3]:
# https://hh.ru/search/vacancy?area=1&fromSearchLine=true&st=searchVacancy&text=%D0%9F%D1%80%D0%BE%D0%B3%D1%80%D0%B0%D0%BC%D0%BC%D0%B8%D1%81%D1%82+python&from=suggest_post&page=0
url = 'https://hh.ru'
params = {'area': 1,
          'fromSearchLine': 'true',
          'st': 'searchVacancy',
          'text': position,
          'page': 0
          }
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) '
                         'Chrome/94.0.4606.71 Safari/537.36 Edg/94.0.992.38'}

In [4]:
url_full = url + '/search/vacancy'

In [5]:
positions = []

In [6]:
df = pd.DataFrame(columns=['website', '_id', 'title', 'employer', 'address', 'salary_min',
                           'salary_max', 'salary_currency', 'link'])

In [7]:
# Функция выборки мин. и макс. зарплаты, если есть
def salary_values(salary_soup):
    if not salary_soup:
        salary_min = None
        salary_max = None
        salary_currency = None
    else:
        salary_list = salary_soup.getText().replace(u'\u202f', u'').split()
        if salary_list[0] == 'от':
            salary_min = int(salary_list[1])
            salary_max = None
        elif salary_list[0] == 'до':
            salary_min = None
            salary_max = int(salary_list[1])
        else:
            salary_min = int(salary_list[0])
            salary_max = int(salary_list[2])
        if len(salary_list) > 3:
            salary_currency = ' '.join(salary_list[3:])
        else:
            salary_currency = salary_list[2]
    return [salary_min, salary_max, salary_currency]

In [8]:
while True:
    response = requests.get(url_full, params=params, headers=headers)
    soup = bs(response.text, 'html.parser')
    position_list = soup.find_all('div', attrs={'class': 'vacancy-serp-item'})

    for position in position_list:
        position_data = {}

        position_info1 = position.find('a', attrs={'data-qa': 'vacancy-serp__vacancy-title'})
        position_info2 = position.find('span', attrs={'data-qa': 'vacancy-serp__vacancy-address'})
        position_info3 = position.find('div', attrs={'class': 'vacancy-serp-item__meta-info-company'})
        position_info4 = position.find('span', attrs={'data-qa': 'vacancy-serp__vacancy-compensation'})

        position_data['website'] = url
        position_data['_id'] = position_info1['href'].split('?')[0].split('/')[-1]
        position_data['title'] = position_info1.getText()
        position_data['employer'] = position_info3.getText()
        position_data['address'] = position_info2.getText()
        position_data['salary_min'] = salary_values(position_info4)[0]
        position_data['salary_max'] = salary_values(position_info4)[1]
        position_data['salary_currency'] = salary_values(position_info4)[2]
        position_data['link'] = position_info1['href']

        positions.append(position_data)

    # переход к следующей странице, если есть
    # Поиск кнопки "дальше"
    button_next = soup.find('a', text='дальше')

    # Проверяем есть ли кнопка "дальше"
    if not button_next or not response.ok:
        print('Страницы закончились')
        # выходим из цикла, если кнопки "дальше" нет
        break
    else:
        # Увеличиваем номер страницы если кнопка "дальше" есть
        params['page'] += 1
        print(f"Переходим на страницу {int(params['page']) + 1}")

Переходим на страницу 2
Переходим на страницу 3
Переходим на страницу 4
Переходим на страницу 5
Переходим на страницу 6
Переходим на страницу 7
Переходим на страницу 8
Переходим на страницу 9
Переходим на страницу 10
Переходим на страницу 11
Переходим на страницу 12
Переходим на страницу 13
Переходим на страницу 14
Переходим на страницу 15
Переходим на страницу 16
Переходим на страницу 17
Переходим на страницу 18
Переходим на страницу 19
Переходим на страницу 20
Переходим на страницу 21
Переходим на страницу 22
Переходим на страницу 23
Переходим на страницу 24
Переходим на страницу 25
Переходим на страницу 26
Переходим на страницу 27
Переходим на страницу 28
Переходим на страницу 29
Переходим на страницу 30
Переходим на страницу 31
Переходим на страницу 32
Переходим на страницу 33
Переходим на страницу 34
Переходим на страницу 35
Переходим на страницу 36
Переходим на страницу 37
Переходим на страницу 38
Переходим на страницу 39
Переходим на страницу 40
Страницы закончились


In [9]:
print(f'Количество собранных вакансий: {len(positions)}')

Количество собранных вакансий: 800


In [10]:
df = df.append(positions)

In [11]:
df[['website', 'title', 'salary_min', 'salary_max', 'salary_currency', 'link']][:10]

,website,title,salary_min,salary_max,salary_currency,link
0,https://hh.ru,Python разработчик,NaN,380000.0,руб.,https://hh.ru/vacancy/47682360?from=vacancy_se...
1,https://hh.ru,Стажер разработчик Python,20000.0,50000.0,руб.,https://hh.ru/vacancy/47861001?from=vacancy_se...
2,https://hh.ru,Junior Data Engineer (Python),120000.0,120000.0,руб.,https://hh.ru/vacancy/47914759?from=vacancy_se...
3,https://hh.ru,Разработчик Python,240000.0,300000.0,руб.,https://hh.ru/vacancy/48652821?from=vacancy_se...
4,https://hh.ru,Разработчик Python / Программист Python (удале...,60000.0,NaN,руб.,https://hh.ru/vacancy/48496694?from=vacancy_se...
5,https://hh.ru,Разработчик Rust/Разработчик Go/Разрабочтик py...,300000.0,NaN,руб.,https://hh.ru/vacancy/48457284?from=vacancy_se...
6,https://hh.ru,Middle Python/Flask developer,180000.0,250000.0,руб.,https://hh.ru/vacancy/48641636?from=vacancy_se...
7,https://hh.ru,"Python-разработчик (Django, DRF)",60000.0,150000.0,руб.,https://hh.ru/vacancy/39736489?from=vacancy_se...
8,https://hh.ru,Python Backend Developer Blockchain,250000.0,450000.0,руб.,https://hh.ru/vacancy/48663627?from=vacancy_se...
9,https://hh.ru,Программист Python (стажер),NaN,NaN,None,https://hh.ru/vacancy/46526734?from=vacancy_se...


In [12]:
# Сохраняем данные в файл формата json
with open('positions.json', 'w') as f:
    json.dump(positions, f, indent=4)
print("Данные сохранены в файл: positions.json")

Данные сохранены в файл: positions.json
